# Porting CUDA programs to HIP

HIP API calls are designed to closely match their CUDA equivalents. This enables HIP to function as a thin layer over CUDA and allows for reasonably easy porting of CUDA code to HIP code. Often it is just a matter of replacing **cuda -> hip** in the function calls. The ROCM library provides two different tools **hipify-perl** and **hipify-clang** to help with the porting process. The tool **hipify-perl** uses perl to perform an intelligent search and replace of cuda calls with hip calls, while the **hipify-clang** tool uses the clang preprocessor to produce a high quality port. The perl-based method is better for quick ports of small codes, while the clang-based method is intended for ports of large codebases, but needs access to all the header files used in the compilation of the CUDA code.

## Setup and installation

From [this source](https://sep5.readthedocs.io/en/latest/Programming_Guides/HIP-porting-guide.html) it is recommended to attempt porting on a machine that has access to both CUDA and HIP libraries. This usually means doing the port on a machine with an NVIDIA GPU. Then one can try porting portions of the code at a time and compare results. For best results with hipify-clang you need to have a version of CUDA that is compatible with your installed version of hipify-clang. 

In [1]:
!hipify-clang --version

AMD LLVM version 16.0.0git
  Optimized build.


Here is a page which describes compatibility between CUDA and hipify-clang.

[HIPIFY Documentation](https://rocm.docs.amd.com/projects/HIPIFY/en/latest/hipify-clang.html)

## Trial setup

There are two sub-directories in this module:

* cuda_mat_mult
* hip_mat_mult

In the directory **cuda_mat_mult** is a CUDA version of the HIP matrix multiplication code in **hip_mat_mult**. It was manually ported from HIP to CUDA. We are going to use the HIP tools to try and port back the CUDA code to HIP code. 

## Porting methods

## Examine the code for porting potential

The hipify-perl tool **hipexamine-perl.sh** and the hipify-clang tool **hipexamine.sh** can both perform a dry run over a directory to see what can be ported.

In [ ]:
!hipexamine 

Available porting tools are **hipify-perl** and **hipify-clang**. Hipify-perl is a perl-based search and replace tool, whereas hipify-clang is a clang preprocessor based tool.


* hipexamine.sh
* hipexamine-perl.sh
* hipconvertinplace.sh
* hipconvertinplace-perl.sh
* hipify-perl
* hipify-clang

### Learnings

* Need to have all the headers available


### Porting with hipify-perl

### Porting with hipify-inplace

## API differences between CUDA and HIP

CUDA has the notion of a driver API and a runtime API.